In [31]:
import pandas as pd
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
raww = pd.read_csv("anime.csv")
raw = pd.read_csv("anime_with_synopsis.csv")
raw = raw.rename(columns={"sypnopsis":"Synopsis"})
raw.head(5)

,MAL_ID,Name,Score,Genres,Synopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [33]:
raw = raw[["MAL_ID","Synopsis"]]
raw.head(6)

,MAL_ID,Synopsis
0,1,"In the year 2071, humanity has colonized sever..."
1,5,"other day, another bounty—such is the life of ..."
2,6,"Vash the Stampede is the man with a $$60,000,0..."
3,7,ches are individuals with special powers like ...
4,8,It is the dark century and the people are suff...
5,15,Sena is like any other shy kid starting high s...


In [34]:
raww = raww[["MAL_ID","Name","Score","Genres","English name","Type","Episodes","Premiered"]]
raww.head(5)

,MAL_ID,Name,Score,Genres,English name,Type,Episodes,Premiered
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,TV,26,Spring 1998
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,Movie,1,Unknown
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,TV,26,Spring 1998
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,TV,26,Summer 2002
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,TV,52,Fall 2004


In [35]:
print(raww.shape)
raw.shape

(17562, 8)


(16214, 2)

In [36]:
df = pd.merge(raw,raww, on="MAL_ID")
df = df[["MAL_ID","Name","Synopsis","Genres","Score","Type","Episodes","Premiered"]]
df.head(5)
df.shape

(16214, 8)

In [37]:
df = df[df["Type"].str.contains("TV")]
df.head()

,MAL_ID,Name,Synopsis,Genres,Score,Type,Episodes,Premiered
0,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","Action, Adventure, Comedy, Drama, Sci-Fi, Space",8.78,TV,26,Spring 1998
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0...","Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",8.24,TV,26,Spring 1998
3,7,Witch Hunter Robin,ches are individuals with special powers like ...,"Action, Mystery, Police, Supernatural, Drama, ...",7.27,TV,26,Summer 2002
4,8,Bouken Ou Beet,It is the dark century and the people are suff...,"Adventure, Fantasy, Shounen, Supernatural",6.98,TV,52,Fall 2004
5,15,Eyeshield 21,Sena is like any other shy kid starting high s...,"Action, Sports, Comedy, Shounen",7.95,TV,145,Spring 2005


In [43]:
df[df.isnull().any(axis=1)]
df = df.dropna()
df.isnull().sum()


Empty DataFrame
Columns: [MAL_ID, Name, Synopsis, Genres, Score, Type, Episodes, Premiered]
Index: []


MAL_ID       0
Name         0
Synopsis     0
Genres       0
Score        0
Type         0
Episodes     0
Premiered    0
dtype: int64

In [44]:
df["Features"] = df["Synopsis"]+df["Genres"]
df.head()

,MAL_ID,Name,Synopsis,Genres,Score,Type,Episodes,Premiered,Features
0,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","Action, Adventure, Comedy, Drama, Sci-Fi, Space",8.78,TV,26,Spring 1998,"In the year 2071, humanity has colonized sever..."
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0...","Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",8.24,TV,26,Spring 1998,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,ches are individuals with special powers like ...,"Action, Mystery, Police, Supernatural, Drama, ...",7.27,TV,26,Summer 2002,ches are individuals with special powers like ...
4,8,Bouken Ou Beet,It is the dark century and the people are suff...,"Adventure, Fantasy, Shounen, Supernatural",6.98,TV,52,Fall 2004,It is the dark century and the people are suff...
5,15,Eyeshield 21,Sena is like any other shy kid starting high s...,"Action, Sports, Comedy, Shounen",7.95,TV,145,Spring 2005,Sena is like any other shy kid starting high s...


In [47]:
#FEauteres to matrix of token counts
cm =CountVectorizer().fit_transform(df["Features"])

In [48]:
cs = cosine_similarity(cm)
print(cs)

[[1.         0.64303454 0.32756054 ... 0.03734279 0.11782556 0.38784248]
 [0.64303454 1.         0.32891836 ... 0.07404587 0.10221429 0.35531634]
 [0.32756054 0.32891836 1.         ... 0.14458881 0.05702659 0.24711524]
 ...
 [0.03734279 0.07404587 0.14458881 ... 1.         0.08451543 0.05634362]
 [0.11782556 0.10221429 0.05702659 ... 0.08451543 1.         0.03333333]
 [0.38784248 0.35531634 0.24711524 ... 0.05634362 0.03333333 1.        ]]


In [58]:
indices = pd.Series(df.Name)
indices[:5]

0          Cowboy Bebop
2                Trigun
3    Witch Hunter Robin
4        Bouken Ou Beet
5          Eyeshield 21
Name: Name, dtype: object

In [67]:
def rec(name,cs=cs):
    recnime =[]
    idx = indices[indices== name].index[0]
    
    score_series=pd.Series(cs[idx]).sort_values(ascending=False)

    top_10_indexes= list(score_series.iloc[1:11].index)

    for i in top_10_indexes:
        recnime.append(list(df.Name)[i])
    return recnime
    print(idx)

In [70]:
rec("Berserk")

IndexError: index 10250 is out of bounds for axis 0 with size 4991